In [1]:
import folium
from folium.features import DivIcon
import geojson

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter

import geopandas as gpd
from shapely import wkt
import shapely as sh

from itertools import chain

import topojson as tp

from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

In [2]:
import branca.colormap as cm

In [3]:
def import_csv(content):
    """Read in csv as a dataframe"""
    acs_data = pd.read_csv(content, encoding= 'unicode_escape')
    return acs_data


def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)

def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist

def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line
    
def to_coords(multipolygon):
    for polygon in multipolygon:
        yield from polygon.exterior.coords[:-1]
        yield from chain.from_iterable(interior.coords[:-1] for interior in polygon.interiors)
        
def strip_dict_keys_values(d):
    """To remove white-space in keys and values in dictionary"""
    def strip_list(l):
        return [strip_dict_keys_values(x)
                if isinstance(x, dict) else strip_list(x)
                if isinstance(x, list) else clean(x) for x in l]

    def clean(value):
        if isinstance(value, str):
            return value.strip()
        return value

    return {key.strip(): strip_dict_keys_values(value)
            if isinstance(value, dict) else strip_list(value)
            if isinstance(value, list) else clean(value)
            if value is None else clean(value)
            for key, value in d.items()}

#get coords for polygon and multipolygon as previous get coord function only worked on polygons. 
def get_coords_from_polygon(shape):
    coords = set()

    if isinstance(shape, sh.geometry.Polygon):
        coords.update(shape.exterior.coords[:-1])
        for linearring in shape.interiors:
            coords.update(linearring.coords[:-1])
    elif isinstance(shape, sh.geometry.MultiPolygon):
        for polygon in shape:
            coords.update(get_coords_from_polygon(polygon))

    return list(coords)

my_tile = 'https://api.mapbox.com/styles/v1/sheldonnb/cklub39iz38pr17mgn9xran5i/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2hlbGRvbm5iIiwiYSI6ImNra3owZzB5ajBidDQycHBsZGY1Zm1vYnAifQ.IiKmYf729J6mYBynizPi2g'

def reversed_colormap(existing):
    return cm.LinearColormap(
        colors=list(reversed(existing.colors)),
        vmin=existing.vmin, vmax=existing.vmax)

In [4]:
wildfire_perimeters = gpd.read_file(r"C:\Users\sheld\Downloads\THESIS_MAPS_\Historic_GeoMAC_Perimeters_Combined_2000-2018-shp")

In [5]:
ACS5 = import_csv(r"C:\Users\sheld\Documents\Thesis_Data_Full_United_States\merged_data_with_block_groups\ACS5_2005_2019_FULL_DATAFRAME_FOR_ANALYSIS_no_geo.csv")

In [6]:
ACS5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129746 entries, 0 to 129745
Data columns (total 92 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         129746 non-null  int64  
 1   GISJOIN            129746 non-null  object 
 2   YEAR               129746 non-null  object 
 3   STATE              129746 non-null  object 
 4   STATEA             129746 non-null  int64  
 5   COUNTY             129746 non-null  object 
 6   COUNTYA            129746 non-null  int64  
 7   TRACTA             129746 non-null  int64  
 8   BLKGRPA            129746 non-null  int64  
 9   NAME_E             129746 non-null  object 
 10  race_0             129746 non-null  int64  
 11  race_1             129746 non-null  float64
 12  race_2             129746 non-null  float64
 13  race_3             129746 non-null  float64
 14  race_4             129746 non-null  float64
 15  race_5             129746 non-null  float64
 16  ra

In [7]:
wildfire_perimeters = wildfire_perimeters[(wildfire_perimeters["state"]== "CA")| (wildfire_perimeters["state"]=="AZ")| (wildfire_perimeters["state"]=="CO")|(wildfire_perimeters["state"]=="ID")|(wildfire_perimeters["state"]=="NE")|(wildfire_perimeters["state"]=="NM")|(wildfire_perimeters["state"]=="OK")|(wildfire_perimeters["state"]=="OR")|(wildfire_perimeters["state"]=="SD")|(wildfire_perimeters["state"]=="TX")|(wildfire_perimeters["state"]=="KS")|(wildfire_perimeters["state"]=="WY")|(wildfire_perimeters["state"]=="MT")|(wildfire_perimeters["state"]=="ND")|(wildfire_perimeters["state"]=="UT")|(wildfire_perimeters["state"]=="NV")|(wildfire_perimeters["state"]=="WA")]

In [8]:
ACS5["POIs"] = list(zip(ACS5.INTPTLAT, ACS5.INTPTLON))

In [9]:
ACS5.medval.min()

6450.190355329949

In [10]:
ACS5.medval.max()

2000001.0

In [33]:
ACS5_2005 = ACS5[ACS5["YEAR"]=="2015-2019"]

In [34]:
ACS5_2005_WEST = ACS5_2005[(ACS5_2005["STATE"] == "Arizona") | (ACS5_2005["STATE"] == "California")|(ACS5_2005["STATE"] == "Colorado") | (ACS5_2005["STATE"] == "Idaho")|(ACS5_2005["STATE"] == "Nebraska")| (ACS5_2005["STATE"] == "New Mexico")|(ACS5_2005["STATE"] == "Oklahoma")| (ACS5_2005["STATE"] == "Oregon")|(ACS5_2005["STATE"] == "South Dakota")|(ACS5_2005["STATE"] == "Texas")|(ACS5_2005["STATE"] == "Kansas")| (ACS5_2005["STATE"] == "Wyoming")|(ACS5_2005["STATE"] == "Montana")|(ACS5_2005["STATE"] == "North Dakota")|(ACS5_2005["STATE"] == "Utah")|(ACS5_2005["STATE"] == "Nevada")| (ACS5_2005["STATE"] == "Washington")].reset_index()

In [35]:
#ACS5_2005_WEST = ACS5_2005_WEST.iloc[::2,:]
#select every second row to reduce map file size

In [36]:
#ACS5_2005_WEST = ACS5_2005_WEST.drop(columns=["Unnamed: 0"]).reset_index()

In [37]:
#get fire years of interest which covers acs 2005-2009 dataframes
fire_years = list(range(2015,2019))
frames = []
for i in range(len(fire_years)):
    wildfires = wildfire_perimeters[wildfire_perimeters["fireyear"] == fire_years[i]]
    frames.append(wildfires)

wildfires_2005_to_2018 = pd.concat(frames,join='inner', ignore_index=True)

In [38]:
fire_min = wildfires_2005_to_2018['gisacres'].min()
fire_max = wildfires_2005_to_2018['gisacres'].max()

linear = cm.LinearColormap(colors=['#ffa64d','#cc0000'],vmin=fire_min,vmax=fire_max)

In [39]:
house_price_min = ACS5_2005_WEST["medval"].min()
house_price_max = ACS5_2005_WEST["medval"].max()

linear_house = cm.LinearColormap(colors=['#1a1aff','#ff80d5','#66ff33','#66ffff','#ffff00','#ffffff'],vmin=house_price_min,vmax=house_price_max)
#linear_house = reversed_colormap(linear_house)

In [40]:
linear

In [41]:
linear_house

In [42]:
m = folium.Map(location=[38.5, -113.158374], zoom_start=5.49, tiles= my_tile, attr = 'XXX Mapbox Attribution', prefer_canvas=True)

title_html = '''
             <h3 align="center" style="font-size:20px"><b>Wildfires 2015-2018 and 2015-2019 Median Property Values by Block Group</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

houses = list(ACS5_2005_WEST["medval"])

for j in range(len(ACS5_2005_WEST["POIs"])):
    poi = ACS5_2005_WEST["POIs"][j]
        
    map_colours_2 = linear_house(houses[j])
    folium.CircleMarker(location=poi, color= map_colours_2, radius=0.000001, fill= map_colours_2).add_to(m)
    
linear_house = linear_house.to_step(index = [6450, 250000, 500000, 800000, 1200000, 1600000, 2000001])
linear_house.caption = 'Median House Prices 2019 Dollars'
linear_house.add_to(m)
    
map_colors = lambda x: {
    'fillColor': linear(x['properties']['gisacres']),
    'color': 'black',
    'weight': 0.25,
    'fillOpacity': 0.5
}
    
#Display Wildfires
folium.GeoJson(
    wildfires_2005_to_2018,
    name = 'Wildfires',
    style_function = map_colors
).add_to(m)
    
folium.LayerControl(collapsed = True).add_to(m)

linear = linear.to_step(index=[0, 100_000, 200_000, 300_000, 400_000, 500_000, 700_000])
linear.caption = 'Number of Acres Burned by Wildfire'
linear.add_to(m)

In [43]:
m.save("2015_2019_Wildfires_US.html")

In [ ]:
#2005-2009, 2010-2014, 2015-2019

In [80]:
linear_house = linear_house.to_step(index = [6450, 300000, 500000, 800000, 1200000, 1500000, 2000001])
linear_house.caption = 'Median House Prices 2019 Dollars'

In [81]:
linear_house